In [1]:
!nvcc --version|

The syntax of the command is incorrect.


In [2]:
!nvidia-smi

Mon Nov 02 19:14:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 2070   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8     6W /  N/A |    578MiB /  8192MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [6]:
!git clone https://github.com/roboflow-ai/darknet.git

Cloning into 'darknet'...


In [1]:
%cd darknet

D:\Datasets\Soft Computing\darknet


In [11]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [2]:
!cat Makefile

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
with open('data/obj.data', 'w') as out:
  out.write('classes = 2\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

In [2]:
#write train file (just the image list)
import os


In [4]:
%cd darknet
with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('./data/obj') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')


D:\ProjectFiles\NumberPlateDetection\darknet


In [5]:
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_darknet.labels')
print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('./cfg/custom-yolov4-detector.cfg'): os.remove('./cfg/custom-yolov4-detector.cfg')


with open('./cfg/custom-yolov4-detector.cfg', 'a') as f:
  f.write('[net]' + '\n')
  f.write('batch=64' + '\n')
  #####smaller subdivisions help the GPU run faster. 12 is optimal, but you might need to change to 24,36,64####
  f.write('subdivisions=24' + '\n')
  f.write('width=416' + '\n')
  f.write('height=416' + '\n')
  f.write('channels=3' + '\n')
  f.write('momentum=0.949' + '\n')
  f.write('decay=0.0005' + '\n')
  f.write('angle=0' + '\n')
  f.write('saturation = 1.5' + '\n')
  f.write('exposure = 1.5' + '\n')
  f.write('hue = .1' + '\n')
  f.write('\n')
  f.write('learning_rate=0.001' + '\n')
  f.write('burn_in=1000' + '\n')
  ######you can adjust up and down to change training time#####
  ##Darknet does iterations with batches, not epochs####
  max_batches = num_classes*2000
  #max_batches = 2000
  f.write('max_batches=' + str(max_batches) + '\n')
  f.write('policy=steps' + '\n')
  steps1 = .8 * max_batches
  steps2 = .9 * max_batches
  f.write('steps='+str(steps1)+','+str(steps2) + '\n')

#Instructions from the darknet repo
#change line classes=80 to your number of objects in each of 3 [yolo]-layers:
#change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.

  with open('cfg/yolov4-custom2.cfg', 'r') as f2:
    content = f2.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 0,1,2' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('cfg/yolov4-custom3.cfg', 'r') as f3:
    content = f3.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 3,4,5' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')

  with open('cfg/yolov4-custom4.cfg', 'r') as f4:
    content = f4.readlines()
    for line in content:
      f.write(line)    
    num_filters = (num_classes + 5) * 3
    f.write('filters='+str(num_filters) + '\n')
    f.write('activation=linear')
    f.write('\n')
    f.write('\n')
    f.write('[yolo]' + '\n')
    f.write('mask = 6,7,8' + '\n')
    f.write('anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401' + '\n')
    f.write('classes=' + str(num_classes) + '\n')
    
  with open('cfg/yolov4-custom5.cfg', 'r') as f5:
    content = f5.readlines()
    for line in content:
      f.write(line)

print("file is written!")    

writing config for a custom YOLOv4 detector detecting number of classes: 2
file is written!


In [6]:
num_classes

2

In [9]:
with open('./cfg/custom-yolov4-detector.cfg', 'a') as f:
    f.write('[net]' + '\n')
    f.write('batch=64' + '\n')
    #####smaller subdivisions help the GPU run faster. 12 is optimal, but you might need to change to 24,36,64####
    f.write('subdivisions=24' + '\n')
    f.write('width=416' + '\n')
    f.write('height=416' + '\n')
    f.write('channels=3' + '\n')
    f.write('momentum=0.949' + '\n')
    f.write('decay=0.0005' + '\n')
    f.write('angle=0' + '\n')
    f.write('saturation = 1.5' + '\n')
    f.write('exposure = 1.5' + '\n')
    f.write('hue = .1' + '\n')
    f.write('\n')
    f.write('learning_rate=0.001' + '\n')
    f.write('burn_in=1000' + '\n')
    ######you can adjust up and down to change training time#####
    ##Darknet does iterations with batches, not epochs####
    max_batches = num_classes*2000
    #max_batches = 2000
    f.write('max_batches=' + str(max_batches) + '\n')
    f.write('policy=steps' + '\n')
    steps1 = .8 * max_batches
    steps2 = .9 * max_batches
    f.write('steps='+str(steps1)+','+str(steps2) + '\n')

In [9]:
!cat cfg/custom-yolov4-detector.cfg

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
!darknet.exe detector train data/obj.data cfg/custom-yolov4-detector.cfg yolov4.conv.137 -dont_show -map

^C
